In [22]:
import pandas as pd
import numpy as np

# Load combined CSV
df = pd.read_csv("crypto_preprocessed.csv")

# Check first 5 rows
print(df.head())

   Unnamed: 0                 Date  BITCOIN_Open  BITCOIN_High   BITCOIN_Low  \
0           0  2024-08-19 05:30:00  58438.293743  58438.293743  58438.293743   
1           1  2024-08-20 05:30:00  59574.945813  59574.945813  59574.945813   
2           2  2024-08-21 05:30:00  59060.126101  59060.126101  59060.126101   
3           3  2024-08-22 05:30:00  61144.687629  61144.687629  61144.687629   
4           4  2024-08-23 05:30:00  60387.540623  60387.540623  60387.540623   

   BITCOIN_Close  BITCOIN_MarketCap  BITCOIN_Volume  ETHEREUM_Open  \
0   58438.293743       1.156217e+12    1.765603e+10    2614.601281   
1   59574.945813       1.176063e+12    2.403133e+10    2640.590421   
2   59060.126101       1.166089e+12    3.147072e+10    2575.643436   
3   61144.687629       1.207099e+12    3.296097e+10    2630.298802   
4   60387.540623       1.192308e+12    2.817030e+10    2623.289171   

   ETHEREUM_High  ...  TETHER_Low  TETHER_Close  TETHER_MarketCap  \
0    2614.601281  ...    0.99

In [6]:
df['Date'] = pd.to_datetime(df['Date'])
df = df.fillna(method='ffill')  # Forward fill
df = df.fillna(method='bfill')  # Backward fill (just in case first rows are NaN)
df.set_index('Date', inplace=True)

C:\Users\harsh\AppData\Local\Temp\ipykernel_9932\1584808300.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')  # Forward fill
C:\Users\harsh\AppData\Local\Temp\ipykernel_9932\1584808300.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')  # Backward fill (just in case first rows are NaN)


In [7]:
df = df[~df.index.duplicated(keep='first')]

In [8]:
features = [
    "BITCOIN_Close", "ETHEREUM_Close", 
    "TETHER_Close", "RIPPLE_Close"
]
data = df[features].values

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [17]:
sequence_length = 30  # last 30 days

X = []
y = []

for i in range(sequence_length, len(data_scaled)):
    X.append(data_scaled[i-sequence_length:i])
    y.append(data_scaled[i])

X, y = np.array(X), np.array(y)
print("X shape:", X.shape, "y shape:", y.shape)

X shape: (339, 30, 4) y shape: (339, 4)


In [18]:
split = int(len(X) * 0.8)  # 80% train, 20% test
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [19]:
df.to_csv("crypto_preprocessed.csv", index=False)
print("✅ Preprocessed CSV saved as 'crypto_preprocessed.csv'")

✅ Preprocessed CSV saved as 'crypto_preprocessed.csv'


In [23]:
features = ["BITCOIN_Close", "ETHEREUM_Close", "TETHER_Close", "RIPPLE_Close"]
data = df[features].values
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [24]:
sequence_length = 30
X, y = [], []
for i in range(sequence_length, len(data_scaled)):
    X.append(data_scaled[i-sequence_length:i])
    y.append(data_scaled[i])
X, y = np.array(X), np.array(y)

In [25]:
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(4))  # 4 outputs: next-day Close for 4 coins

model.compile(optimizer='adam', loss='mse')
model.summary()

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

C:\Users\harsh\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 30, 64)              │          17,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 30, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 30,212 (118.02 KB)

 Trainable params: 30,212 (118.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - loss: 0.0879 - val_loss: 0.0363
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0279 - val_loss: 0.0269
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0205 - val_loss: 0.0240
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0170 - val_loss: 0.0140
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0162 - val_loss: 0.0131
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0151 - val_loss: 0.0103
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0157 - val_loss: 0.0107
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0135 - val_loss: 0.0101
Epoch 9/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step - loss: 0.0139 - val_loss: 0.0110
Epoch 10/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0119 - val_loss: 0.0116
Epoch 11/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0131 - val_loss: 0.0142
Epoch 12/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0

In [30]:
model.save("multi_coin_lstm.keras")

In [31]:
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 274ms/step


In [35]:
from sklearn.preprocessing import MinMaxScaler

features = ["BITCOIN_Close", "ETHEREUM_Close", "TETHER_Close", "RIPPLE_Close"]
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df[features].values)

# Save this scaler
import pickle
with open("scaler_close.save", "wb") as f:
    pickle.dump(scaler, f)